# *openQmin* script generator

This generates *runHelper.py* automatically from the command-line arguments defined in *openQmin.cpp*.

In [13]:
with open('../openQmin.cpp','r') as f:
    openQmincpp = f.readlines()
arglines = [ line for line in openQmincpp if 'ValueArg' in line ]    

pythonlines = ["""
\"\"\"
This Python script simply calls openQmin.out in command-line mode, 
with all available command-line arguments presented as a Python dictionary.
Variables with empty-string values are ignored, as are commented-out lines. 
\"\"\"

import os

mpi_num_processes = 1

params = dict("""]

for line in arglines[1:]:
    shortname, longname, hint, _, default, vartype = [ eval(item)
        for item in ''.join(
            ''.join(
                line.split('(')[1:]
            ).split(')')[:-1]
        ).replace('false','False').replace('true', 'True').split(',')[:-1]
    ]
    if vartype == 'scalar':
        vartype = 'float'
    elif vartype == 'string':
        default = '\"\"'
    pythonline = f'\t{longname} = {default}, # <{vartype}> {hint}'
    pythonlines.append(pythonline)
pythonlines.append(')')
pythonscript = '\n'.join(pythonlines)

pythonscript += """

def get_runcmd():
    runcmd = '../build/openQmin.out ' + ' '.join([
        f'--{key} {val}' for key, val in zip(
            params.keys(), params.values()
        )
        if len(str(val)) > 0
    ])
    if mpi_num_processes > 1:
        runcmd = f'mpirun -n {mpi_num_processes} ' + runcmd
    return runcmd
    
if __name__ == '__main__':
    runcmd = get_runcmd()
    print(runcmd)
    os.system(runcmd)
"""

with open ('runHelper.py', 'w') as f:
    f.write(pythonscript)